# What you can learn about food by analyzing a million Yelp reviews

#### Based on https://github.com/skipgram/modern-nlp-in-python
#### Updated to Python 3 wherever possible

_Note: Patrick Harrison presented the original notebook this one was based on as a tutorial during the [PyData DC 2016 conference](http://pydata.org/dc2016/schedule/presentation/11/). To view the video of the presentation on YouTube, see [here](https://www.youtube.com/watch?v=6zm9NC9uRkk)._

## Our Trail Map
This tutorial features an end-to-end data science & natural language processing pipeline, starting with **raw data** and running through **preparing**, **modeling**, **visualizing**, and **analyzing** the data. We'll touch on the following points:
1. A tour of the dataset
1. Introduction to text processing with spaCy
1. Automatic phrase modeling
1. Topic modeling with LDA
1. Visualizing topic models with pyLDAvis
1. Word vector models with word2vec
1. Visualizing word2vec with t-SNE

...and we might even learn a thing or two about Python along the way.

Let's get started!

## The Yelp Dataset
[**The Yelp Dataset**](https://www.yelp.com/dataset_challenge/) is a dataset published by the business review service [Yelp](http://yelp.com) for academic research and educational purposes. I really like the Yelp dataset as a subject for machine learning and natural language processing demos, because it's big (but not so big that you need your own data center to process it), well-connected, and anyone can relate to it &mdash; it's largely about food, after all!

**Note:** If you'd like to execute this notebook interactively on your local machine, you'll need to download your own copy of the Yelp dataset. If you're reviewing a static copy of the notebook online, you can skip this step. Here's how to get the dataset:
1. Please visit the Yelp dataset webpage [here](https://www.yelp.com/dataset_challenge/)
1. Click "Get the Data"
1. Please review, agree to, and respect Yelp's terms of use!
1. The dataset downloads as a compressed .tgz file; uncompress it
1. Place the uncompressed dataset files (*yelp_academic_dataset_business.json*, etc.) in a directory named *yelp_dataset_challenge_academic_dataset*
1. Place the *yelp_dataset_challenge_academic_dataset* within the *data* directory in the *Modern NLP in Python* project folder

That's it! You're ready to go.

The current iteration of the Yelp dataset (as of this demo) consists of the following data:
- __552K__ users
- __77K__ businesses
- __2.2M__ user reviews

When focusing on restaurants alone, there are approximately __22K__ restaurants with approximately __1M__ user reviews written about them.

The data is provided in a handful of files in _.json_ format. We'll be using the following files for our demo:
- __yelp\_academic\_dataset\_business.json__ &mdash; _the records for individual businesses_
- __yelp\_academic\_dataset\_review.json__ &mdash; _the records for reviews users wrote about businesses_

The files are text files (UTF-8) with one _json object_ per line, each one corresponding to an individual data record. Let's take a look at a few examples.

In [26]:
!ls ../data/yelp_dataset_challenge_academic_dataset/dataset

business.json checkin.json  review.json   tip.json      user.json


In [4]:
import shutil
# tar_src_file = os.path.join('..', 'data', 'yelp_dataset.tar')
# tar_target = os.path.join(tar_src,'yelp_dataset_challenge_academic_dataset')
#shutil.unpack_archive(tar_src_file, extract_dir=tar_target)

In [3]:
import os
import codecs


data_directory = os.path.join('..', 'data',
                              'yelp_dataset_challenge_academic_dataset', 'dataset')
businesses_filepath = os.path.join(data_directory,
                                   'business.json')
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print(first_business_record)

{"business_id": "YDf95gJZaq05wvo7hTQbbQ", "name": "Richmond Town Square", "neighborhood": "", "address": "691 Richmond Rd", "city": "Richmond Heights", "state": "OH", "postal_code": "44143", "latitude": 41.5417162, "longitude": -81.4931165, "stars": 2.0, "review_count": 17, "is_open": 1, "attributes": {"RestaurantsPriceRange2": 2, "BusinessParking": {"garage": false, "street": false, "validated": false, "lot": true, "valet": false}, "BikeParking": true, "WheelchairAccessible": true}, "categories": ["Shopping", "Shopping Centers"], "hours": {"Monday": "10:00-21:00", "Tuesday": "10:00-21:00", "Friday": "10:00-21:00", "Wednesday": "10:00-21:00", "Thursday": "10:00-21:00", "Sunday": "11:00-18:00", "Saturday": "10:00-21:00"}}



In [5]:
!ls ../data/yelp_dataset_challenge_academic_dataset/dataset

business.json checkin.json  review.json   tip.json      user.json


In [ ]:
# I ran out of memory, so the full files were possibly not downloaded.. 

{"business_id": "vcNAWiLM4dR7D2nwwJ7nCA", "full_address": "4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018", "hours": {"Tuesday": {"close": "17:00", "open": "08:00"}, "Friday": {"close": "17:00", "open": "08:00"}, "Monday": {"close": "17:00", "open": "08:00"}, "Wednesday": {"close": "17:00", "open": "08:00"}, "Thursday": {"close": "17:00", "open": "08:00"}}, "open": true, "categories": ["Doctors", "Health & Medical"], "city": "Phoenix", "review_count": 9, "name": "Eric Goldberg, MD", "neighborhoods": [], "longitude": -111.98375799999999, "state": "AZ", "stars": 3.5, "latitude": 33.499313000000001, "attributes": {"By Appointment Only": true}, "type": "business"}



The business records consist of _key, value_ pairs containing information about the particular business. A few attributes we'll be interested in for this demo include:
- __business\_id__ &mdash; _unique identifier for businesses_
- __categories__ &mdash; _an array containing relevant category values of businesses_

The _categories_ attribute is of special interest. This demo will focus on restaurants, which are indicated by the presence of the _Restaurant_ tag in the _categories_ array. In addition, the _categories_ array may contain more detailed information about restaurants, such as the type of food they serve.

The review records are stored in a similar manner &mdash; _key, value_ pairs containing information about the reviews.

In [6]:
review_filepath = os.path.join(data_directory,
                                    'review.json')

with codecs.open(review_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
print(first_review_record)

{"review_id":"VfBHSwC5Vz_pbFluy07i9Q","user_id":"cjpdDjZyprfyDG3RlkVG3w","business_id":"uYHaNptLzDLoV_JZ_MuzUA","stars":5,"date":"2016-07-12","text":"My girlfriend and I stayed here for 3 nights and loved it. The location of this hotel and very decent price makes this an amazing deal. When you walk out the front door Scott Monument and Princes street are right in front of you, Edinburgh Castle and the Royal Mile is a 2 minute walk via a close right around the corner, and there are so many hidden gems nearby including Calton Hill and the newly opened Arches that made this location incredible.\n\nThe hotel itself was also very nice with a reasonably priced bar, very considerate staff, and small but comfortable rooms with excellent bathrooms and showers. Only two minor complaints are no telephones in room for room service (not a huge deal for us) and no AC in the room, but they have huge windows which can be fully opened. The staff were incredible though, letting us borrow umbrellas for t

A few attributes of note on the review records:
- __business\_id__ &mdash; _indicates which business the review is about_
- __text__ &mdash; _the natural language text the user wrote_

The _text_ attribute will be our focus today!

_json_ is a handy file format for data interchange, but it's typically not the most usable for any sort of modeling work. Let's do a bit more data preparation to get our data in a more usable format. Our next code block will do the following:
1. Read in each business record and convert it to a Python `dict`
2. Filter out business records that aren't about restaurants (i.e., not in the "Restaurant" category)
3. Create a `frozenset` of the business IDs for restaurants, which we'll use in the next step

In [28]:
# import json

# restaurant_ids = set()

# # open the businesses file
# with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
#     # iterate through each line (json record) in the file
#     for business_json in f:
        
#         # convert the json record to a Python dict
#         business = json.loads(business_json)
        
#         # if this business is not a restaurant, skip to the next one
#         if u'Restaurants' not in business[u'categories']:
#             continue
            
#         # add the restaurant business id to our restaurant_ids set
#         restaurant_ids.add(business[u'business_id'])

# # turn restaurant_ids into a frozenset, as we don't need to change it anymore
# restaurant_ids = frozenset(restaurant_ids)

# # print the number of unique restaurant ids in the dataset
# print('There are {} restaurants in the dataset.'.format(len(restaurant_ids)))

There are 51613 restaurants in the dataset.


In [7]:
!ls ../data/yelp_dataset_challenge_academic_dataset/dataset

business.json checkin.json  review.json   tip.json      user.json


Next, we will create a new directory that contains only the text from reviews about restaurants, with one review per line in the file.

In [32]:
data_directory

'../data/yelp_dataset_challenge_academic_dataset/dataset'

In [8]:
intermediate_directory = os.path.join('..','data','yelp_dataset_challenge_academic_dataset', 'extracted_from_json')

review_txt_filepath = os.path.join(intermediate_directory,
                                   'review_text_all.txt')

review_json_filepath = os.path.join(data_directory,'review.json')

In [42]:
#!mkdir ../data/yelp_dataset_challenge_academic_dataset/extracted_from_json

In [9]:
!ls ../data/yelp_dataset_challenge_academic_dataset/extracted_from_json

review_text_all.txt


In [10]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.

if False:
    
    review_count = 0

    # create & open a new file in write mode
    with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with codecs.open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant, skip to the next one
                if review[u'business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print ('Text from {} restaurant reviews written to the new txt file.'.format(review_count))
    
else:
    
    with codecs.open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print('Text from {} restaurant reviews in the txt file.'.format(review_count + 1))

Text from 2929512 restaurant reviews in the txt file.
CPU times: user 46.6 s, sys: 1.7 s, total: 48.3 s
Wall time: 50.7 s


## spaCy &mdash; Industrial-Strength NLP in Python

![spaCy](https://s3.amazonaws.com/skipgram-images/spaCy.png)

[**spaCy**](https://spacy.io) is an industrial-strength natural language processing (_NLP_) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.

spaCy handles many tasks commonly associated with building an end-to-end natural language processing pipeline:
- Tokenization
- Text normalization, such as lowercasing, stemming/lemmatization
- Part-of-speech tagging
- Syntactic dependency parsing
- Sentence boundary detection
- Named entity recognition and annotation

In the "batteries included" Python tradition, spaCy contains built-in data and models which you can use out-of-the-box for processing general-purpose English language text:
- Large English vocabulary, including stopword lists
- Token "probabilities"
- Word vectors

spaCy is written in optimized Cython, which means it's _fast_. According to a few independent sources, it's the fastest syntactic parser available in any language. Key pieces of the spaCy parsing pipeline are written in pure C, enabling efficient multithreading (i.e., spaCy can release the _GIL_).

In [11]:
# !python -m spacy download en_core_web_md



    100% |████████████████████████████████| 1083.2MB 11.2MB/s ta 0:00:01  2% |▊                               | 23.4MB 27.7MB/s eta 0:00:39    7% |██▍                             | 80.6MB 21.0MB/s eta 0:00:48    10% |███▍                            | 113.3MB 22.2MB/s eta 0:00:44    13% |████▏                           | 142.0MB 24.5MB/s eta 0:00:39    14% |████▋                           | 157.2MB 41.0MB/s eta 0:00:23    17% |█████▌                          | 185.1MB 37.9MB/s eta 0:00:24    | 212.3MB 9.0MB/s eta 0:01:37K    19% |██████▍                         | 216.4MB 39.5MB/s eta 0:00:22              | 239.4MB 61.4MB/s eta 0:00:14    23% |███████▍                        | 251.9MB 29.8MB/s eta 0:00:2823% |███████▌                        | 254.1MB 41.3MB/s eta 0:00:21��███▋                        | 256.1MB 44.8MB/s eta 0:00:19    24% |███████▊                        | 261.6MB 21.5MB/s eta 0:00:39    27% |████████▉                       | 300.2MB 13.8MB/s eta 0:00:57 42.1MB/s eta 0:0

In [14]:
# !python -m spacy link en_core_web_md en_default


    Linking successful

    /anaconda/lib/python3.6/site-packages/en_core_web_md/en_core_web_md-1.2.1
    --> /anaconda/lib/python3.6/site-packages/spacy/data/en_default

    You can now load the model via spacy.load('en_default').



In [16]:
import spacy
import pandas as pd
import itertools as it
nlp = spacy.load('en_default')

Let's grab a sample review to play with.

In [17]:
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

The staff here is great and they're nice,  wonderful and quick. People were ranting in raving about pei wei, I had to try it.  Even good yelp reviews.  I'm highly dissatisfied with the flavor of the food. This  should be labeled Asian inspired and not Asian. I've tried a variety of Chinese restaurants, this doesn't taste close to anything I've had at other Asian restaurants. Their Mongolian beef  was 5 pieces of beef and large mushrooms cut into thirds in a thick sauce. You eat the rice to wash off the nasty flavor. My shrimp was thickly coated in an overpowering  sauce as well.  I only ate some of the veggies that take center stage on a meat dish.  The center of my pork egg roll was cold. The hot N sour soup was a much thicker consistency almost like that of a chili instead of being brothy. Worst of all was the price.  This was not worth it to us. Neither me or my husband enjoyed either of  our dishes.  We didn't even eat half of our plates.  We even refused to take it home with us.  

Hand the review text to spaCy, and be prepared to wait...

In [18]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 37.7 ms, sys: 33.5 ms, total: 71.1 ms
Wall time: 74.8 ms


...1/20th of a second or so. Let's take a look at what we got during that time...

In [19]:
print(parsed_review)

The staff here is great and they're nice,  wonderful and quick. People were ranting in raving about pei wei, I had to try it.  Even good yelp reviews.  I'm highly dissatisfied with the flavor of the food. This  should be labeled Asian inspired and not Asian. I've tried a variety of Chinese restaurants, this doesn't taste close to anything I've had at other Asian restaurants. Their Mongolian beef  was 5 pieces of beef and large mushrooms cut into thirds in a thick sauce. You eat the rice to wash off the nasty flavor. My shrimp was thickly coated in an overpowering  sauce as well.  I only ate some of the veggies that take center stage on a meat dish.  The center of my pork egg roll was cold. The hot N sour soup was a much thicker consistency almost like that of a chili instead of being brothy. Worst of all was the price.  This was not worth it to us. Neither me or my husband enjoyed either of  our dishes.  We didn't even eat half of our plates.  We even refused to take it home with us.  

Looks the same! What happened under the hood?

What about sentence detection and segmentation?

In [20]:
for num, sentence in enumerate(parsed_review.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
The staff here is great and they're nice,  wonderful and quick.

Sentence 2:
People were ranting in raving about pei wei

Sentence 3:
, I had to try it.  

Sentence 4:
Even good yelp reviews.  

Sentence 5:
I'm highly dissatisfied with the flavor of the food.

Sentence 6:
This  should be labeled Asian inspired and not Asian.

Sentence 7:
I've tried a variety of Chinese restaurants, this doesn't taste close to anything I've had at other Asian restaurants.

Sentence 8:
Their Mongolian beef  was 5 pieces of beef and large mushrooms cut into thirds in a thick sauce.

Sentence 9:
You eat the rice to wash off the nasty flavor.

Sentence 10:
My shrimp was thickly coated in an overpowering  sauce as well.  

Sentence 11:
I only ate some of the veggies that take center stage on a meat dish.  

Sentence 12:
The center of my pork egg roll was cold.

Sentence 13:
The hot N sour soup was a much thicker consistency almost like that of a chili instead of being brothy.

Sentence 14:
Worst 

What about named entity detection?

In [24]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    #print ''

Entity 1: Asian - NORP
Entity 2: Asian - NORP
Entity 3: Chinese - NORP
Entity 4: Asian - NORP
Entity 5: Mongolian - NORP
Entity 6: 5 - CARDINAL
Entity 7: half - CARDINAL
Entity 8: Asian - NORP


What about part of speech tagging?

In [27]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]



<class 'list'> <class 'list'>


In [32]:
pd.DataFrame(list(zip(token_text, token_pos)),columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,The,DET
1,staff,NOUN
2,here,ADV
3,is,VERB
4,great,ADJ
5,and,CCONJ
6,they,PRON
7,'re,VERB
8,nice,ADJ
9,",",PUNCT


What about text normalization, like stemming/lemmatization and shape analysis?

In [33]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,The,the,Xxx
1,staff,staff,xxxx
2,here,here,xxxx
3,is,be,xx
4,great,great,xxxx
5,and,and,xxx
6,they,-PRON-,xxxx
7,'re,be,'xx
8,nice,nice,xxxx
9,",",",",","


What about token-level entity analysis?

In [34]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,The,,O
1,staff,,O
2,here,,O
3,is,,O
4,great,,O
5,and,,O
6,they,,O
7,'re,,O
8,nice,,O
9,",",,O


What about a variety of other token-level attributes, such as the relative frequency of tokens, and whether or not a token matches any of these categories?
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [35]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_review]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,The,-5.958707,,,,,
1,staff,-10.435253,,,,,
2,here,-7.068025,,,,,
3,is,-4.457749,,,,,
4,great,-7.420455,,,,,
5,and,-4.113108,,,,,
6,they,-5.524368,,,,,
7,'re,-6.255463,,,,,
8,nice,-8.009807,,,,,
9,",",-3.454960,,Yes,,,


If the text you'd like to process is general-purpose English language text (i.e., not domain-specific, like medical literature), spaCy is ready to use out-of-the-box.

I think it will eventually become a core part of the Python data science ecosystem &mdash; it will do for natural language computing what other great libraries have done for numerical computing.

## Phrase Modeling Using Gensim

_Phrase modeling_ is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our reviews and looking for words that _co-occur_ (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} * N > threshold$$

...where:
* $count(A)$ is the number of times token $A$ appears in the corpus
* $count(B)$ is the number of times token $B$ appears in the corpus
* $count(A\ B)$ is the number of times the tokens $A\ B$ appear in the corpus *in order*
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
* $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so _new york_ would become *new\_york*). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as _happy hour_) to also become phrases in the model.

We turn to the indispensible [**gensim**](https://radimrehurek.com/gensim/index.html) library to help us with phrase modeling &mdash; the [**Phrases**](https://radimrehurek.com/gensim/models/phrases.html) class in particular.

In [38]:
# !conda install -y gensim

Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /anaconda:

The following NEW packages will be INSTALLED:

    bz2file:    0.98-py36_0                  
    gensim:     2.3.0-np112py36_0            
    smart_open: 1.5.3-py36_0      conda-forge

bz2file-0.98-p 100% |################################| Time: 0:00:00  12.14 MB/s
smart_open-1.5 100% |################################| Time: 0:00:00 269.65 kB/s
gensim-2.3.0-n 100% |################################| Time: 0:00:00  14.33 MB/s


In [39]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

As we're performing phrase modeling, we'll be doing some iterative data transformation at the same time. Our roadmap for data preparation includes:

1. Segment text of complete reviews into sentences & normalize text
1. First-order phrase modeling $\rightarrow$ _apply first-order phrase model to transform sentences_
1. Second-order phrase modeling $\rightarrow$ _apply second-order phrase model to transform sentences_
1. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

First, let's define a few helper functions that we'll use for text normalization. In particular, the `lemmatized_sentence_corpus` generator function will use spaCy to:
- Iterate over the 1M reviews in the `review_txt_all.txt` we created before
- Segment the reviews into individual sentences
- Remove punctuation and excess whitespace
- Lemmatize the text

... and do so efficiently in parallel, thanks to spaCy's `nlp.pipe()` function.

In [ ]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=-1):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [ ]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all_2.txt')

Let's use the `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text. We'll write this data back out to a new file (`unigram_sentences_all`), with one normalized sentence per line. We'll use this data for learning our phrase models.

In [42]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')

KeyboardInterrupt: 

If your data is organized like our `unigram_sentences_all` file now is &mdash; a large text file with one document/sentence per line &mdash; gensim's [**LineSentence**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) class provides a convenient iterator for working with other gensim components. It *streams* the documents/sentences from disk, so that you never have to hold the entire corpus in RAM at once. This allows you to scale your modeling pipeline up to potentially very large corpora.

In [43]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

Let's take a look at a few sample sentences in our new, transformed file.

In [45]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print(' '.join(unigram_sentence))
    print('')

-PRON- wife want to try this place for awhile now

one of -PRON- brother friend use to be a manager of one in colorado too

tonight -PRON- decide to try -PRON- out

-PRON- place -PRON- order online

easy process

-PRON- wife go with an order of chicken lo mein and -PRON- go with sesame chicken steak with fried rice as well as an egg roll and 2 pot sticker

total for everything be $ 24.80 after tax

-PRON- drive down and pick up -PRON- food

the manager -PRON- be friendly ring -PRON- up real quick and -PRON- be out the door in no time

this bag be heavy as hell too



Next, we'll learn a phrase model that will link individual words into two-word phrases. We'd expect words that together represent a specific concept, like "`ice cream`", to be linked together to form a new, single token: "`ice_cream`".

In [46]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [47]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if True:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 57.7 s, sys: 1.72 s, total: 59.5 s
Wall time: 1min 1s


Now that we have a trained phrase model for word pairs, let's apply it to the review sentences data and explore the results.

In [48]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')

In [49]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 2min 2s, sys: 1.68 s, total: 2min 4s
Wall time: 2min 8s


In [50]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [51]:
for bigram_sentence in it.islice(bigram_sentences, 230, 240):
    print(' '.join(bigram_sentence))
    print('')

-PRON- wife want to try this place for awhile now

one of -PRON- brother friend use to be a manager of one in colorado too

tonight -PRON- decide to try -PRON- out

-PRON- place -PRON- order online

easy process

-PRON- wife go with an order of chicken lo_mein and -PRON- go with sesame chicken steak with fried_rice as well as an egg roll and 2 pot_sticker

total for everything be $ 24.80 after tax

-PRON- drive down and pick_up -PRON- food

the manager -PRON- be friendly ring -PRON- up real quick and -PRON- be out the door in no time

this bag be heavy as hell too



In [52]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

In [53]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if False:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

CPU times: user 56.1 s, sys: 1.23 s, total: 57.3 s
Wall time: 59.1 s


We'll apply our trained second-order phrase model to our first-order transformed sentences, write the results out to a new file, and explore a few of the second-order transformed sentences.

In [54]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')

In [55]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if False:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1min 58s, sys: 1.59 s, total: 1min 59s
Wall time: 2min 3s


In [56]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [59]:
for trigram_sentence in it.islice(trigram_sentences, 280, 510):
    print(' '.join(trigram_sentence))
    print('')

haven't eat at this location in particular but in general the price be high for what -PRON- get

-PRON- be nothing_special and any other chinese place in the city can probably beat -PRON- in the price quality area

always on point

-PRON- get the carmel chicken

the crab_wonton be a guilt must have

and -PRON- love there tea too

eat -PRON- for two year and always pleased

thx

-PRON- love pei_wei

the food be delicious

the atmosphere be chill

the restaurant be clean and staff friendly

came_here with -PRON- friend today cause -PRON- be starve and decide to go whichever next place -PRON- see

and -PRON- be very surprised at how pretty the inside of the restaurant look

the decor be amazing

-PRON- order the japanese teryaki bowl vegetarian style and -PRON- be amazing

-PRON- love the tofu and the brown_rice and the veggie

-PRON- friend order the lo_mein and -PRON- love -PRON-

-PRON- do not finish due to the fact that the portion be huge

-PRON- do not know -PRON- be a like share th

The final step of our text preparation process circles back to the complete text of the reviews. We're going to run the complete text of the reviews through a pipeline that applies our text normalization and phrase models.

In addition, we'll remove stopwords at this point. _Stopwords_ are very common words, like _a_, _the_, _and_, and so on, that serve functional roles in natural language, but typically don't contribute to the overall meaning of text. Filtering stopwords is a common procedure that allows higher-level NLP modeling techniques to focus on the words that carry more semantic weight.

Finally, we'll write the transformed text out to a new file, with one review per line.

In [60]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [67]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=2):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


KeyboardInterrupt: 

In [66]:
spacy.en.STOP_WORDS

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'front',
 'full',
 'further',
 'get',
 'gi

Let's preview the results. We'll grab one review from the file with the original, untransformed text, grab the same review from the file with the normalized and transformed text, and compare the two.

In [70]:
print('Original: \n')

for review in it.islice(line_review(review_txt_filepath), 11, 12):
    print(review)

print('----\n')
print('Transformed:\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print(review)

Original: 

I love this place i'd recommend it to anyone ! We always order it togo and it never disappoints! The food always taste fresh and is always ready on time! Definitely our favorite lunch spot !

----

Transformed:

-PRON- love place -PRON- recommend -PRON- -PRON- order -PRON- togo -PRON- disappoint food taste fresh ready time definitely -PRON- favorite lunch spot



You can see that most of the grammatical structure has been scrubbed from the text &mdash; capitalization, articles/conjunctions, punctuation, spacing, etc. However, much of the general semantic *meaning* is still present. Also, multi-word concepts such as "`friday_night`" and "`above_average`" have been joined into single tokens, as expected. The review text is now ready for higher-level modeling. 

## Topic Modeling with Latent Dirichlet Allocation (_LDA_)

*Topic modeling* is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) or LDA, a popular approach to topic modeling.

In many conventional NLP applications, documents are represented a mixture of the individual tokens (words and phrases) they contain. In other words, a document is represented as a *vector* of token counts. There are two layers in this model &mdash; documents and tokens &mdash; and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
* Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)
* They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.
* The dimensions are fully indepedent from each other &mdash; there's no sense of connection between related tokens, such as _knife_ and _fork_.

LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of *topics*, and the *topics* are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

![LDA](https://s3.amazonaws.com/skipgram-images/LDA.png)

LDA is fully unsupervised. The topics are "discovered" automatically from the data by trying to maximize the likelihood of observing the documents in your corpus, given the modeling assumptions. They are expected to capture some latent structure and organization within the documents, and often have a meaningful human interpretation for people familiar with the subject material.

We'll again turn to gensim to assist with data preparation and modeling. In particular, gensim offers a high-performance parallelized implementation of LDA with its [**LdaMulticore**](https://radimrehurek.com/gensim/models/ldamulticore.html) class.

In [74]:
!pip install pyldavis

    100% |████████████████████████████████| 1.6MB 380kB/s ta 0:00:011
    100% |████████████████████████████████| 184kB 2.4MB/s ta 0:00:011
    100% |████████████████████████████████| 829kB 523kB/s ta 0:00:011
  Running setup.py bdist_wheel for pyldavis ... - \ done
  Stored in directory: /Users/seanreed1/Library/Caches/pip/wheels/de/41/af/cba16e4c15ff942728f3345c8f165831b03ad7f4d87cff8b6e
  Running setup.py bdist_wheel for future ... - \ | / done
  Stored in directory: /Users/seanreed1/Library/Caches/pip/wheels/c2/50/7c/0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
  Running setup.py bdist_wheel for funcy ... - \ done
  Stored in directory: /Users/seanreed1/Library/Caches/pip/wheels/0a/7e/0f/7e6946d9291cf9963926645a1dee5dd5a9c522daa5a659e0a2
Successfully built pyldavis future funcy


In [78]:
#!pip install cPickle

  Could not find a version that satisfies the requirement cPickle (from versions: )
No matching distribution found for cPickle


In [75]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
#import cPickle as pickle

ModuleNotFoundError: No module named 'cPickle'

In [ ]:
https://join.slack.com/t/python-and-datasci/shared_invite/enQtMjU3MDcwNTcxNjIxLTAyZjY0MzMwMGUyNGVmNGMwNDdkMGUyNTg2MzBkYWYxZGE1MzA4OTMyNTI5MTI3YmZiMjM3MGUxY2E4NWQ0Mjg

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) class for this.

In [79]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [80]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if False:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 11.5 s, sys: 177 ms, total: 11.7 s
Wall time: 12.2 s


Like many NLP techniques, LDA uses a simplifying assumption known as the [*bag-of-words* model](https://en.wikipedia.org/wiki/Bag-of-words_model). In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded. 

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The `trigram_bow_generator` function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In the following code, "bag-of-words" is abbreviated as `bow`.

In [81]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [82]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [84]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if True:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 16.8 s, sys: 392 ms, total: 17.1 s
Wall time: 17.8 s


With the bag-of-words corpus, we're finally ready to learn our topic model from the reviews. We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to `LdaMulticore` as inputs, along with the number of topics the model should learn. For this demo, we're asking for 30 topics.

In [85]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [118]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if True:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=1)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 5min 51s, sys: 43.7 s, total: 6min 35s
Wall time: 5min 48s


Our topic model is now trained and ready to use! Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [92]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('term             frequency')

    for term, frequency in lda.show_topic(topic_number, topn=topn):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [119]:
explore_topic(topic_number=0)

term             frequency
like                 0.045
pretty               0.025
taste                0.019
think                0.013
look                 0.011
eat                  0.010
bad                  0.009
smell                0.009
little               0.008
try                  0.008
airport              0.007
use                  0.007
price                0.007
cheap                0.007
thing                0.006
's                   0.006
decent               0.006
lot                  0.006
actually             0.006
know                 0.006
buck                 0.006
want                 0.005
meh                  0.005
maybe                0.005
people               0.005


In [120]:
explore_topic(topic_number=1)

term             frequency
bouncer              0.035
da                   0.023
man                  0.023
sweet_tea            0.014
oder                 0.014
hier                 0.013
essen                0.013
das_essen            0.012
alle                 0.012
sangria              0.012
nur                  0.010
uns                  0.010
falafel              0.009
hatte                0.009
breakfast_burrito    0.008
mehr                 0.007
gibt_es              0.007
voucher              0.007
diese                0.007
das_ambiente         0.007
black_dog            0.007
haben                0.007
bedienung            0.006
wedding              0.006
panera               0.006


In [121]:
explore_topic(topic_number=2)

term             frequency
restaurant           0.029
dish                 0.020
order                0.017
try                  0.016
service              0.016
come                 0.016
chinese              0.014
fresh                0.013
great                0.013
definitely           0.012
price                0.012
delicious            0.012
pho                  0.012
love                 0.011
time                 0.011
lunch                0.010
friendly             0.010
spicy                0.010
noodle               0.010
thai                 0.010
rice                 0.008
eat                  0.008
clean                0.008
find                 0.008
korean               0.008


In [122]:
explore_topic(topic_number=3)

term             frequency
buffet               0.159
selection            0.027
lunch                0.024
dessert              0.021
vegas                0.018
station              0.017
george               0.017
variety              0.013
section              0.012
quality              0.011
d                    0.010
chinatown            0.010
eat                  0.009
cheesesteak          0.009
vs                   0.008
dinner               0.007
item                 0.006
ppl                  0.006
try                  0.006
pretty               0.006
western              0.005
like                 0.005
asian                0.005
shrimp_cocktail      0.004
filler               0.004


In [123]:
explore_topic(topic_number=4)

term             frequency
security             0.023
kimchi               0.021
banchan              0.012
ayce                 0.010
office               0.010
coworker             0.009
service              0.009
blt                  0.007
highly               0.007
table                0.007
korean               0.006
restaurant           0.006
wear                 0.006
future               0.005
club                 0.005
djs                  0.005
delay                0.005
self                 0.005
speak_english        0.005
exit                 0.005
hassle               0.004
hire                 0.004
facility             0.004
maintain             0.004
wave                 0.004


In [124]:
explore_topic(topic_number=5)

term             frequency
hummus               0.019
gyro                 0.019
+                    0.015
greek                0.014
pita                 0.012
fajita               0.010
menu                 0.010
salad                0.009
like                 0.009
shave_ice            0.008
philly               0.008
takoyaki             0.008
mediterranean        0.008
order                0.007
platter              0.007
madison              0.007
fresh                0.007
tasty                0.006
small                0.006
greek_salad          0.005
try                  0.005
delicious            0.005
cucumber             0.005
middle_eastern       0.004
plate                0.004


In [125]:
explore_topic(topic_number=6)

term             frequency
great                0.102
service              0.057
friendly             0.032
staff                0.030
time                 0.021
excellent            0.021
love                 0.020
atmosphere           0.019
nice                 0.018
amazing              0.018
delicious            0.017
definitely           0.015
restaurant           0.012
experience           0.012
come                 0.011
awesome              0.011
attentive            0.011
enjoy                0.010
dinner               0.010
menu                 0.009
server               0.009
fantastic            0.009
recommend            0.009
visit                0.009
wonderful            0.008


In [126]:
explore_topic(topic_number=7)

term             frequency
burger               0.161
fry                  0.053
come                 0.016
shake                0.016
great                0.014
try                  0.014
bun                  0.013
delicious            0.011
love                 0.011
onion_ring           0.010
order                0.010
like                 0.008
patty                0.008
definitely           0.007
bacon                0.007
milkshake            0.007
chuck_box            0.007
eat                  0.007
juicy                0.007
cook                 0.007
time                 0.006
vig                  0.006
n                    0.006
want                 0.006
vegas                0.006


In [127]:
explore_topic(topic_number=8)

term             frequency
pretty               0.016
xs                   0.015
like                 0.015
bf                   0.013
come                 0.012
groupon              0.011
filet                0.010
okay                 0.010
bit_pricey           0.009
$_18                 0.009
teriyaki             0.007
pistachio            0.006
tho                  0.006
kalua_pig            0.006
japanese             0.006
try                  0.006
nice                 0.006
teriyaki_chicken     0.006
=)                   0.005
spin                 0.005
pull_pork_sandwich   0.005
seven                0.005
uncle                0.005
whip_cream           0.005
pretty_darn          0.005


In [128]:
explore_topic(topic_number=9)

term             frequency
order                0.048
time                 0.032
come                 0.030
wait                 0.028
service              0.025
table                0.021
ask                  0.018
server               0.015
drink                0.013
waitress             0.012
restaurant           0.010
bring                0.009
leave                0.008
seat                 0.008
experience           0.008
tell                 0.008
check                0.007
arrive               0.007
bad                  0.007
long                 0.007
busy                 0.007
waiter               0.007
meal                 0.007
water                0.006
finally              0.006


In [129]:
explore_topic(topic_number=10)

term             frequency
meat                 0.024
wrap                 0.020
korean_bbq           0.015
like                 0.013
grill                0.013
bulgogi              0.011
crab_leg             0.011
burrito              0.011
eat                  0.010
mango                0.010
little               0.009
soda                 0.008
2                    0.008
green                0.007
3                    0.007
potato_salad         0.006
sticky_rice          0.006
bean                 0.006
juice                0.006
rice                 0.006
cilantro             0.006
time                 0.006
add                  0.006
macaroni_salad       0.006
try                  0.006


In [130]:
explore_topic(topic_number=11)

term             frequency
friend               0.021
indian               0.013
restaurant           0.010
's                   0.009
like                 0.008
share                0.008
feel                 0.007
enjoy                0.007
nice                 0.007
dessert              0.007
play                 0.006
table                0.006
montreal             0.006
tea                  0.006
board_game           0.006
chocolate            0.006
crab_puff            0.005
group                0.005
child                0.005
think                0.005
menu                 0.005
space                0.005
lovely               0.005
love                 0.005
time                 0.005


In [131]:
explore_topic(topic_number=12)

term             frequency
breakfast            0.052
order                0.017
cafe                 0.016
coffee               0.015
time                 0.015
day                  0.013
lunch                0.011
morning              0.011
home                 0.011
pastry               0.010
wait                 0.008
diner                0.008
potato               0.008
come                 0.008
counter              0.008
love                 0.007
work                 0.007
fruit                0.006
egg                  0.006
delicious            0.006
fresh                0.006
eat                  0.006
today                0.006
omelet               0.006
friendly             0.006


In [132]:
explore_topic(topic_number=13)

term             frequency
salad                0.056
soup                 0.037
lunch                0.023
like                 0.019
option               0.018
try                  0.014
menu                 0.014
vegetarian           0.011
sandwich             0.011
love                 0.010
fresh                0.010
find                 0.009
look                 0.008
cookie               0.008
great                0.008
healthy              0.007
day                  0.007
lot                  0.007
veggie               0.006
offer                0.006
eat                  0.006
want                 0.006
choice               0.006
tasty                0.006
delicious            0.006


In [133]:
explore_topic(topic_number=14)

term             frequency
dish                 0.022
try                  0.016
menu                 0.014
like                 0.014
bbq                  0.013
dessert              0.013
rib                  0.011
restaurant           0.011
delicious            0.011
mac                  0.011
sweet                0.010
taste                0.010
flavor               0.010
bit                  0.009
cheese               0.008
definitely           0.008
meal                 0.007
dinner               0.007
great                0.007
sauce                0.006
flavour              0.006
tender               0.006
love                 0.006
rich                 0.006
pork_belly           0.006


In [134]:
explore_topic(topic_number=15)

term             frequency
korean               0.054
duck                 0.022
culinary_dropout     0.019
reasonable_price     0.019
gut                  0.016
udon                 0.010
flavour              0.010
favourite            0.010
pretentious          0.010
kbbq                 0.010
wird                 0.009
restaurant           0.008
superb               0.008
edinburgh            0.008
qdoba                0.007
und_das              0.007
soju                 0.007
bubble_tea           0.007
-the                 0.006
chef                 0.006
mojito               0.006
queen                0.006
schnitzel            0.006
makai                0.006
amazed               0.006


The first topic has strong associations with words like *taco*, *salsa*, *chip*, *burrito*, and *margarita*, as well as a handful of more general words. You might call this the **Mexican food** topic!

It's possible to go through and inspect each topic in the same way, and try to assign a human-interpretable label that captures the essence of each one. I've given it a shot for all 50 topics below.

In [47]:
# topic_names = {0: u'mexican',
#                1: u'menu',
#                2: u'thai',
#                3: u'steak',
#                4: u'donuts & appetizers',
#                5: u'specials',
#                6: u'soup',
#                7: u'wings, sports bar',
#                8: u'foreign language',
#                9: u'las vegas',
#                10: u'chicken',
#                11: u'aria buffet',
#                12: u'noodles',
#                13: u'ambience & seating',
#                14: u'sushi',
#                15: u'arizona',
#                16: u'family',
#                17: u'price',
#                18: u'sweet',
#                19: u'waiting',
#                20: u'general',
#                21: u'tapas',
#                22: u'dirty',
#                23: u'customer service',
#                24: u'restrooms',
#                25: u'chinese',
#                26: u'gluten free',
#                27: u'pizza',
#                28: u'seafood',
#                29: u'amazing',
#                30: u'eat, like, know, want',
#                31: u'bars',
#                32: u'breakfast',
#                33: u'location & time',
#                34: u'italian',
#                35: u'barbecue',
#                36: u'arizona',
#                37: u'indian',
#                38: u'latin & cajun',
#                39: u'burger & fries',
#                40: u'vegetarian',
#                41: u'lunch buffet',
#                42: u'customer service',
#                43: u'taco, ice cream',
#                44: u'high cuisine',
#                45: u'healthy',
#                46: u'salad & sandwich',
#                47: u'greek',
#                48: u'poor experience',
#                49: u'wine & dine'}

In [48]:
topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'w') as f:
    pickle.dump(topic_names, f)

You can see that, along with **mexican**, there are a variety of topics related to different styles of food, such as **thai**, **steak**, **sushi**, **pizza**, and so on. In addition, there are topics that are more related to the overall restaurant *experience*, like **ambience & seating**, **good service**, **waiting**, and **price**.

Beyond these two categories, there are still some topics that are difficult to apply a meaningful human interpretation to, such as topic 30 and 43.

Manually reviewing the top terms for each topic is a helpful exercise, but to get a deeper understanding of the topics and how they relate to each other, we need to visualize the data &mdash; preferably in an interactive format. Fortunately, we have the fantastic [**pyLDAvis**](https://pyldavis.readthedocs.io/en/latest/readme.html) library to help with that!

pyLDAvis includes a one-line function to take topic models created with gensim and prepare their data for visualization.

In [135]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [136]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

#     with open(LDAvis_data_filepath, 'w') as f:
#         pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath) as f:
#     LDAvis_prepared = pickle.load(f)

/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


CPU times: user 7min 2s, sys: 3.98 s, total: 7min 6s
Wall time: 4min 18s


`pyLDAvis.display(...)` displays the topic model visualization in-line in the notebook.

In [137]:
pyLDAvis.display(LDAvis_prepared)

### Wait, what am I looking at again?
There are a lot of moving parts in the visualization. Here's a brief summary:

* On the left, there is a plot of the "distance" between all of the topics (labeled as the _Intertopic Distance Map_)
  * The plot is rendered in two dimensions according a [*multidimensional scaling (MDS)*](https://en.wikipedia.org/wiki/Multidimensional_scaling) algorithm. Topics that are generally similar should be appear close together on the plot, while *dis*similar topics should appear far apart.
  * The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.
  * An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.
* On the right, there is a bar chart showing top terms.
  * When no topic is selected in the plot on the left, the bar chart shows the top-30 most "salient" terms in the corpus. A term's *saliency* is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.
  * When a particular topic is selected, the bar chart changes to show the top-30 most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter $\lambda$, which can be adjusted with a slider above the bar chart.
    * Setting the $\lambda$ parameter close to 1.0 (the default) will rank the terms solely according to their probability within the topic.
    * Setting $\lambda$ close to 0.0 will rank the terms solely according to their "distinctiveness" or "exclusivity" within the topic &mdash; i.e., terms that occur *only* in this topic, and do not occur in other topics.
    * Setting $\lambda$ to values between 0.0 and 1.0 will result in an intermediate ranking, weighting term probability and exclusivity accordingly.
* Rolling the mouse over a term in the bar chart on the right will cause the topic circles to resize in the plot on the left, to show the strength of the relationship between the topics and the selected term.

A more detailed explanation of the pyLDAvis visualization can be found [here](https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf). Unfortunately, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics. If you need to match up topics in gensim's `LdaMulticore` object and pyLDAvis' visualization, you have to dig through the terms manually.

### Analyzing our LDA model
The interactive visualization pyLDAvis produces is helpful for both:
1. Better understanding and interpreting individual topics, and
1. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the $\lambda$ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

For (2), exploring the _Intertopic Distance Plot_ can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

In our plot, there is a stark divide along the x-axis, with two topics far to the left and most of the remaining 48 far to the right. Inspecting the two outlier topics provides a plausible explanation: both topics contain many non-English words, while most of the rest of the topics are in English. So, one of the main attributes that distinguish the reviews in the dataset from one another is their language.

This finding isn't entirely a surprise. In addition to English-speaking cities, the Yelp dataset includes reviews of businesses in Montreal and Karlsruhe, Germany, often written in French and German, respectively. Multiple languages isn't a problem for our demo, but for a real NLP application, you might need to ensure that the text you're processing is written in English (or is at least tagged for language) before passing it along to some downstream processing. If that were the case, the divide along the x-axis in the topic plot would immediately alert you to a potential data quality issue.

The y-axis separates two large groups of topics &mdash; let's call them "super-topics" &mdash; one in the upper-right quadrant and the other in the lower-right quadrant. These super-topics correlate reasonably well with the pattern we'd noticed while naming the topics:
* The super-topic in the *lower*-right tends to be about *food*. It groups together the **burger & fries**, **breakfast**, **sushi**, **barbecue**, and **greek** topics, among others.
* The super-topic in the *upper*-right tends to be about other elements of the *restaurant experience*. It groups together the **ambience & seating**, **location & time**, **family**, and **customer service** topics, among others.

So, in addition to the 50 direct topics the model has learned, our analysis suggests a higher-level pattern in the data. Restaurant reviewers in the Yelp dataset talk about two main things in their reviews, in general: (1) the food, and (2) their overall restaurant experience. For this dataset, this is a very intuitive result, and we probably didn't need a sophisticated modeling technique to tell it to us. When working with datasets from other domains, though, such high-level patterns may be much less obvious from the outset &mdash; and that's where topic modeling can help.

### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% _Topic A_, 20% _Topic B_, 20% _Topic C_, and 10% _Topic D_.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:
1. Using spaCy to remove punctuation and lemmatize the text
1. Applying our first-order phrase model to join word pairs
1. Applying our second-order phrase model to join longer phrases
1. Removing stopwords
1. Creating a bag-of-words representation

Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The `lda_description(...)` function will perform all these steps for us, including printing the resulting topical description of the input text.

In [52]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [53]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.en.STOPWORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda (topic_number, freq): -freq)
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print '{:25} {}'.format(topic_names[topic_number],
                                round(freq, 3))

In [54]:
sample_review = get_sample_review(50)
print sample_review

Best French toast ever!! Love the friendly atmosphere, and especially the breakfast. Never been disappointed. You have to try French toast with raisin bread too... yummy!



In [55]:
lda_description(sample_review)

breakfast                 0.4
amazing                   0.341
customer service          0.192


In [56]:
sample_review = get_sample_review(100)
print sample_review

I went there for dinner last night with a client.  This is second time I visited.  I had a scotch and he had a Guiness.  The (-1) is for drink selection. Just stock some better beers and higher end scotch and you're five stars.

We started with the meatballs covered with Provolone and other blessed goodness.  These did not disappoint.  I had a four cheese pizza with sweet sausage and garlic.  It was fantastic.  They have so many good dishes, but I wanted the pizza last night.  I couldn't finish the pizza - way to go big medium pizza.

I finished up with a coffee.  The parking can be a bit of a challenge on the street, but it's a small town atmosphere in Carnegie, PA.  I love the downtown there.



In [57]:
lda_description(sample_review)

pizza                     0.28
bars                      0.193
amazing                   0.172
wine & dine               0.149


## Word Vector Embedding with Word2Vec

Pop quiz! Can you complete this text snippet?

<br><br>

![word2vec quiz](https://s3.amazonaws.com/skipgram-images/word2vec-1.png)

<br><br><br>
You just demonstrated the core machine learning concept behind word vector embedding models!
<br><br><br>

![word2vec quiz 2](https://s3.amazonaws.com/skipgram-images/word2vec-2.png)

The goal of *word vector embedding models*, or *word vector models* for short, is to learn dense, numerical vector representations for each term in a corpus vocabulary. If the model is successful, the vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary. Word vector models are also fully unsupervised &mdash; they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.

Perhaps the best-known word vector model is [word2vec](https://arxiv.org/pdf/1301.3781v3.pdf), originally proposed in 2013. The general idea of word2vec is, for a given *focus word*, to use the *context* of the word &mdash; i.e., the other words immediately before and after it &mdash; to provide hints about what the focus word might mean. To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.

At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. The model then slides the window across every snippet of text in the corpus, with each word taking turns as the focus word. Each time the model considers a new snippet, it tries to learn some information about the focus word based on the surrouding context, and it "nudges" the words' vector representations accordingly. One complete pass sliding the window across all of the corpus text is known as a training *epoch*. It's common to train a word2vec model for multiple passes/epochs over the corpus. Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are *close* to each other in vector space.

For a deeper dive into word2vec's machine learning process, see [here](https://arxiv.org/pdf/1411.2738v4.pdf).

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors. Typical choices include a few dozen to several hundred.
- The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
- The number of training epochs.

For using word2vec in Python, [gensim](https://rare-technologies.com/deep-learning-with-word2vec-and-gensim/) comes to the rescue again! It offers a [highly-optimized](https://rare-technologies.com/word2vec-in-python-part-two-optimizing/), [parallelized](https://rare-technologies.com/parallelizing-word2vec-in-python/) implementation of the word2vec algorithm with its [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) class.

In [58]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

We'll train our word2vec model using the normalized sentences with our phrase models applied. We'll use 100-dimensional vectors, and set up our training process to run for twelve epochs.

In [59]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 0 == 1:

    # initiate the model and perform the first epoch of training
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        food2vec.train(trigram_sentences)
        food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print u'{} training epochs so far.'.format(food2vec.train_count)

12 training epochs so far.
CPU times: user 5.43 s, sys: 891 ms, total: 6.32 s
Wall time: 7.12 s


On my four-core machine, each epoch over all the text in the ~1 million Yelp reviews takes about 5-10 minutes.

In [60]:
print u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.vocab))

50,835 terms in the food2vec vocabulary.


Let's take a peek at the word vectors our model has learned. We'll create a pandas DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [90]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.vocab.iteritems()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda (term, index, count): -count)

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,-0.035762,-0.173890,-0.035782,-0.007144,0.032371,-0.065272,-0.219383,-0.064665,0.002739,0.025802,...,0.050136,0.044030,0.145281,-0.020442,0.128879,-0.076461,0.075532,-0.012841,0.024710,-0.067555
be,-0.074780,-0.049524,0.085974,-0.098892,0.141556,0.024878,-0.011119,-0.175374,0.005410,-0.110996,...,-0.199047,-0.081284,-0.198344,0.007257,0.075339,0.070266,-0.008326,-0.127542,-0.046246,0.110279
and,-0.070505,-0.026918,0.028344,-0.099909,0.127974,-0.058155,-0.056091,-0.028973,0.197281,-0.040528,...,-0.049051,-0.212434,-0.042576,0.055731,0.117097,-0.206737,0.055435,-0.065056,0.052316,-0.078666
i,-0.161238,0.050831,-0.081706,-0.084479,0.053073,-0.102327,-0.108607,-0.001920,-0.057367,-0.050715,...,0.028528,-0.016578,-0.179229,0.053357,0.070913,0.036893,-0.000544,-0.007254,-0.056005,0.106345
a,-0.083491,-0.033712,-0.124125,-0.110776,-0.033046,-0.089950,0.025416,-0.052321,-0.059281,0.074985,...,-0.101939,0.022392,0.057049,0.015819,-0.001798,0.001103,0.003096,0.037175,-0.074279,0.001683
to,-0.012082,0.033135,-0.063183,-0.057252,-0.018721,-0.017931,-0.027784,0.112110,0.020549,-0.174336,...,-0.017111,-0.067532,-0.022149,0.154788,-0.093789,-0.020456,0.065478,0.075484,-0.053530,-0.005314
it,0.025022,0.081581,0.127987,-0.188015,0.041450,-0.126222,0.172725,-0.149931,-0.069566,-0.036031,...,0.045720,0.094828,0.089329,0.051623,-0.108989,-0.145476,0.068617,0.090687,-0.101725,0.090377
have,-0.140812,-0.070552,0.022102,0.001077,0.109890,-0.061365,0.046450,0.003073,0.113845,-0.038957,...,-0.051071,-0.090922,-0.022011,0.157082,-0.082406,-0.010306,-0.063481,-0.098728,-0.064020,0.153466
of,-0.036341,-0.054903,0.000644,-0.010602,0.168195,-0.058505,-0.052342,0.039159,-0.053572,-0.160039,...,0.085908,-0.211464,-0.084990,0.082315,0.223018,-0.142501,0.280647,0.003435,-0.037710,-0.145140
not,-0.075276,0.109047,0.055135,0.052251,0.209437,0.084334,-0.122419,-0.193307,0.000699,-0.099067,...,-0.150619,-0.060446,0.181940,-0.118538,-0.002879,0.018827,0.084586,0.040437,0.070277,-0.047521


Holy wall of numbers! This DataFrame has 50,835 rows &mdash; one for each term in the vocabulary &mdash; and 100 colums. Our model has learned a quantitative vector representation for each term, as expected.

Put another way, our model has "embedded" the terms into a 100-dimensional vector space.

### So... what can we do with all these numbers?
The first thing we can use them for is to simply look up related words and phrases for a given term of interest.

In [63]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print u'{:20} {}'.format(word, round(similarity, 3))

### What things are like Burger King?

In [64]:
get_related_terms(u'burger_king')

mcdonalds            0.895
wendy_'s             0.855
mcd_'s               0.853
mcdonald_'s          0.852
denny_'s             0.816
bk                   0.808
carl_'s_jr.          0.8
red_robin            0.792
mickey_d_'s          0.771
sonic                0.765


The model has learned that fast food restaurants are similar to each other! In particular, *mcdonalds* and *wendy's* are the most similar to Burger King, according to this dataset. In addition, the model has found that alternate spellings for the same entities are probably related, such as *mcdonalds*, *mcdonald's* and *mcd's*.

### When is happy hour?

In [65]:
get_related_terms(u'happy_hour')

hh                   0.874
reverse_happy_hour   0.801
happy_hr             0.771
during_happy_hour    0.672
mon_fri              0.634
3_6pm                0.632
hh.                  0.631
4_7pm                0.625
special              0.621
happy_hour_3_6pm     0.618


The model has noticed several alternate spellings for happy hour, such as *hh* and *happy hr*, and assesses them as highly related. If you were looking for reviews about happy hour, such alternate spellings would be very helpful to know.

Taking a deeper look &mdash; the model has turned up phrases like *3-6pm*, *4-7pm*, and *mon-fri*, too. This is especially interesting, because the model has no advance knowledge at all about what happy hour is, and what time of day it should be. But simply by scanning through restaurant reviews, the model has discovered that the concept of happy hour has something very important to do with that block of time around 3-7pm on weekdays.

### Let's make pasta tonight. Which style do you want?

In [66]:
get_related_terms(u'pasta', topn=20)

lasagna              0.798
spaghetti            0.773
bolognese            0.757
fettuccine           0.748
penne                0.745
rigatoni             0.743
angel_hair           0.721
linguine             0.716
angel_hair_pasta     0.712
penne_pasta          0.712
carbonara            0.705
fettucini            0.704
alfredo              0.703
tortellini           0.703
manicotti            0.7
ziti                 0.698
gnocci               0.694
ravioli              0.694
linguini             0.693
risotto              0.691


## Word algebra!
No self-respecting word2vec demo would be complete without a healthy dose of *word algebra*, also known as *analogy completion*.

The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure goes like this:
1. Provide a set of words or phrases that you'd like to add or subtract.
1. Look up the vectors that represent those terms in the word vector model.
1. Add and subtract those vectors to produce a new, combined vector.
1. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
1. Return the word(s) associated with the similar vector(s).

But more generally, you can think of the vectors that represent each word as encoding some information about the *meaning* or *concepts* of the word. What happens when you ask the model to combine the meaning and concepts of words in new ways? Let's see.

In [67]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print term

### breakfast + lunch = ?
Let's start with a softball.

In [68]:
word_algebra(add=[u'breakfast', u'lunch'])

brunch


OK, so the model knows that *brunch* is a combination of *breakfast* and *lunch*. What else?

### lunch - day + night = ?

In [69]:
word_algebra(add=[u'lunch', u'night'], subtract=[u'day'])

dinner


Now we're getting a bit more nuanced. The model has discovered that:
- Both *lunch* and *dinner* are meals
- The main difference between them is time of day
- Day and night are times of day
- Lunch is associated with day, and dinner is associated with night

What else?

### taco - mexican + chinese = ?

In [70]:
word_algebra(add=[u'taco', u'chinese'], subtract=[u'mexican'])

dumpling


Here's an entirely new and different type of relationship that the model has learned.
- It knows that tacos are a characteristic example of Mexican food
- It knows that Mexican and Chinese are both styles of food
- If you subtract *Mexican* from *taco*, you're left with something like the concept of a *"characteristic type of food"*, which is represented as a new vector
- If you add that new *"characteristic type of food"* vector to Chinese, you get *dumpling*.

What else?

### bun - american + mexican = ?

In [71]:
word_algebra(add=[u'bun', u'mexican'], subtract=[u'american'])

tortilla


The model knows that both *buns* and *tortillas* are the doughy thing that goes on the outside of your real food, and that the primary difference between them is the style of food they're associated with.

What else?

### filet mignon - beef + seafood = ?

In [72]:
word_algebra(add=[u'filet_mignon', u'seafood'], subtract=[u'beef'])

raw_oyster


The model has learned a concept of *delicacy*. If you take filet mignon and subtract beef from it, you're left with a vector that roughly corresponds to delicacy. If you add the delicacy vector to *seafood*, you get *raw oyster*.

What else?

### coffee - drink + snack = ?

In [73]:
word_algebra(add=[u'coffee', u'snack'], subtract=[u'drink'])

pastry


The model knows that if you're on your coffee break, but instead of drinking something, you're eating something... that thing is most likely a pastry.

What else?

### Burger King + fine dining = ?

In [74]:
word_algebra(add=[u'burger_king', u'fine_dining'])

denny_'s


Touché. It makes sense, though. The model has learned that both Burger King and Denny's are large chains, and that both serve fast, casual, American-style food. But Denny's has some elements that are slightly more upscale, such as printed menus and table service. Fine dining, indeed.

*What if we keep going?*

### Denny's + fine dining = ?

In [75]:
word_algebra(add=[u"denny_'s", u'fine_dining'])

applebee_'s


This seems like a good place to land... what if we explore the vector space around *Applebee's* a bit, in a few different directions? Let's see what we find.

#### Applebee's + italian = ?

In [76]:
word_algebra(add=[u"applebee_'s", u'italian'])

olive_garden


#### Applebee's + pancakes = ?

In [77]:
word_algebra(add=[u"applebee_'s", u'pancakes'])

ihop


#### Applebee's + pizza = ?

In [78]:
word_algebra(add=[u"applebee_'s", u'pizza'])

pizza_hut


You could do this all day. One last analogy before we move on...

### wine - grapes + barley = ?

In [79]:
word_algebra(add=[u'wine', u'barley'], subtract=[u'grapes'])

beer


## Word Vector Visualization with t-SNE

[t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf), or *t-SNE* for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

scikit-learn provides a convenient implementation of the t-SNE algorithm with its [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class.

In [80]:
from sklearn.manifold import TSNE

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first:
1. Drop stopwords &mdash; it's probably not too interesting to visualize *the*, *of*, *or*, and so on
1. Take only the 5,000 most frequent terms in the vocabulary &mdash; no need to visualize all ~50,000 terms right now.

In [81]:
tsne_input = word_vectors.drop(spacy.en.STOPWORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

In [82]:
tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
food,-0.164133,0.007745,0.058311,-0.169839,-0.042278,0.004095,0.203732,-0.021252,-0.084491,-0.016372,...,-0.096869,0.060159,-0.133541,0.166804,0.084901,0.109261,0.137871,0.018093,-0.158754,-0.042917
good,-0.239592,-0.232940,-0.005036,-0.028226,0.149816,-0.133312,-0.034164,-0.130310,-0.013757,0.008618,...,-0.034984,-0.135347,-0.112965,0.056312,0.055106,-0.026181,-0.135510,0.087664,0.009934,-0.111619
place,0.025479,0.130311,0.119834,-0.096365,0.013793,0.074431,-0.063780,0.063191,-0.004273,0.111458,...,-0.076251,-0.076574,-0.086146,-0.023936,0.136419,-0.001543,-0.084301,0.016356,-0.148379,-0.016498
order,0.045996,-0.035101,-0.045906,-0.280336,0.157393,-0.146304,-0.064311,-0.082754,0.124693,-0.194072,...,-0.063306,0.125928,-0.194433,-0.129551,0.039680,0.058868,-0.023189,-0.153715,0.152482,-0.003842
great,-0.189524,-0.259744,-0.041182,0.021664,0.132266,-0.030005,-0.078055,-0.004111,-0.016184,0.054305,...,-0.052275,-0.122914,-0.075555,0.193383,0.017008,-0.117156,-0.041065,0.140430,-0.067349,-0.040291


In [83]:
tsne_filepath = os.path.join(intermediate_directory,
                             u'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     u'tsne_vectors.npy')

In [93]:
%%time

if 0 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'w') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath) as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

CPU times: user 31.3 ms, sys: 2.47 ms, total: 33.8 ms
Wall time: 32.9 ms


Now we have a two-dimensional representation of our data! Let's take a look.

In [94]:
tsne_vectors.head()

,x_coord,y_coord
food,2.313886,6.475995
good,8.763030,4.633407
place,-8.942178,2.221976
order,-2.876029,-2.300830
great,9.515772,5.076319


In [95]:
tsne_vectors[u'word'] = tsne_vectors.index

### Plotting with Bokeh

In [88]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [89]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, resize, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

## Conclusion

Whew! Let's round up the major components that we've seen:
1. Text processing with **spaCy**
1. Automated **phrase modeling**
1. Topic modeling with **LDA** $\ \longrightarrow\ $ visualization with **pyLDAvis**
1. Word vector modeling with **word2vec** $\ \longrightarrow\ $ visualization with **t-SNE**

#### Why use these models?
Dense vector representations for text like LDA and word2vec can greatly improve performance for a number of common, text-heavy problems like:
- Text classification
- Search
- Recommendations
- Question answering

...and more generally are a powerful way machines can help humans make sense of what's in a giant pile of text. They're also often useful as a pre-processing step for many other downstream machine learning applications.

## Data Science @ S&P Global &mdash; *we are hiring!*